In [100]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,precision_score, recall_score, f1_score,confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler
import customtkinter as cus
import tkinter as tk
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

KHÁM PHÁ DỮ LIỆU

In [101]:
# Đọc và xem kích thước dữ liệu
train_file_path = 'train_dataset.csv'
test_file_path = 'test_dataset.csv'

train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)

train_data.shape

(237, 14)

In [102]:
# Hiển thị 5 dòng đầu dữ liệu
train_data.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,54.0,1.0,0,124.0,266.0,0.0,2.0,109.0,1.0,2.2,2.0,1.0,2,1
1,46.0,1.0,2,101.0,197.0,1.0,0.0,156.0,0.0,0.0,1.0,0.0,2,0
2,43.0,1.0,1,130.0,315.0,0.0,0.0,162.0,0.0,1.9,1.0,1.0,1,0
3,47.0,1.0,1,138.0,257.0,0.0,2.0,156.0,0.0,0.0,1.0,0.0,1,0
4,58.0,1.0,0,150.0,270.0,0.0,2.0,111.0,1.0,0.8,1.0,0.0,2,1


In [103]:
# Kiểm tra dữ liệu NAN (giá trị thiếu)
train_data.isna().sum()

Age          0
Sex          0
ChestPain    0
RestBP       0
Chol         0
Fbs          0
RestECG      0
MaxHR        0
ExAng        0
Oldpeak      0
Slope        0
Ca           0
Thal         0
AHD          0
dtype: int64

CHUẨN HÓA DỮ LIỆU

In [104]:
# Xử lý dữ liệu (chuẩn hóa và xử lý nếu cần)
def preprocess_data(data):
    X = data.drop(columns=['AHD'])  # Loại bỏ cột mục tiêu
    y = data['AHD']                # Lấy cột mục tiêu
    scaler = StandardScaler()      # Chuẩn hóa dữ liệu
    X_scaled = scaler.fit_transform(X)
    
    # Chuyển đổi X_scaled thành DataFrame
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
    
    return X_scaled, y

In [105]:
# Tách dữ liệu huấn luyện và kiểm tra
X_train, Y_train = preprocess_data(train_data)
X_test, y_test = preprocess_data(test_data)

In [106]:
# Hàm tính lỗi trung bình
def error(y, y_pred):
    return np.mean(y != y_pred)

# KFold Cross Validation (chỉ sử dụng tập huấn luyện)
kf = KFold(n_splits=10, random_state=None, shuffle=True)

# Lưu kết quả từ các fold
validation_results = []
test_results = []

min1, min2, min3 = float('inf'), float('inf'), float('inf')
id3_clf1 = per_clf1 = noron_clf1 = None

for train_index, validation_index in kf.split(X_train):
    X_train_split, X_validation = X_train.iloc[train_index], X_train.iloc[validation_index]
    y_train_split, y_validation = Y_train.iloc[train_index], Y_train.iloc[validation_index]

    # Khởi tạo các mô hình
    per_clf = Perceptron(alpha=0.01, max_iter=1000)
    per_clf.fit(X_train_split, y_train_split)

    id3_clf = DecisionTreeClassifier(criterion="entropy", max_depth=10, min_samples_split=5, random_state=1)
    id3_clf.fit(X_train_split, y_train_split)

    noron_clf = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=1)
    noron_clf.fit(X_train_split, y_train_split)

    # Tính lỗi và cập nhật mô hình tốt nhất
    y_val_pred1 = id3_clf.predict(X_validation)
    y_val_pred2 = per_clf.predict(X_validation)
    y_val_pred3 = noron_clf.predict(X_validation)

    id3_sum_error = error(y_validation, y_val_pred1)
    per_sum_error = error(y_validation, y_val_pred2)
    noron_sum_error = error(y_validation, y_val_pred3)

    if id3_sum_error < min1:
        min1 = id3_sum_error
        id3_clf1 = id3_clf
    if per_sum_error < min2:
        min2 = per_sum_error
        per_clf1 = per_clf
    if noron_sum_error < min3:
        min3 = noron_sum_error
        noron_clf1 = noron_clf

# VotingClassifier to combine the models
voting_clf = VotingClassifier(
    estimators=[
        ('id3', id3_clf1),
        ('per', per_clf1),
        ('noron', noron_clf1)
    ],
    voting='hard'  # Hard voting
)

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

In [107]:
# Huấn luyện mô hình VotingClassifier
voting_clf.fit(X_train, Y_train)

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


VotingClassifier(estimators=[('id3',
                              DecisionTreeClassifier(criterion='entropy',
                                                     max_depth=10,
                                                     min_samples_split=5,
                                                     random_state=1)),
                             ('per', Perceptron(alpha=0.01)),
                             ('noron', MLPClassifier(random_state=1))])

In [108]:
# Dự đoán trên tập test
y_test_pred = voting_clf.predict(X_test)

In [112]:
# Đánh giá trên tập test
test_results = {
    "accuracy": accuracy_score(y_test, y_test_pred),
    "precision": precision_score(y_test, y_test_pred),
    "recall": recall_score(y_test, y_test_pred),
    "f1": f1_score(y_test, y_test_pred)
}

# Kết quả
print("Kết quả đánh giá trên tập test:")
print(f"Accuracy: {test_results['accuracy']:.2f}, Precision: {test_results['precision']:.2f}, Recall: {test_results['recall']:.2f}, F1-Score: {test_results['f1']:.2f}")

Kết quả đánh giá trên tập test:
Accuracy: 0.88, Precision: 0.92, Recall: 0.82, F1-Score: 0.87


In [111]:
# Đánh giá mô hình trên tập test
y_test_pred = voting_clf.predict(X_test)
accuracy = np.mean(y_test == y_test_pred)

# Giao diện người dùng
root = cus.CTk()
root.title("Phần mềm dự đoán bệnh tim")

width, height = 800, 600
x = (root.winfo_screenwidth() - width) // 2
y = (root.winfo_screenheight() - height) // 2
root.geometry(f"{width}x{height}+{x}+{y}")

app_title = cus.CTkLabel(root, text="Phần mềm dự đoán bệnh tim", font=("Arial", 18, "bold"), text_color="blue")
app_title.pack(pady=10)

# info_label = cus.CTkLabel(
#     root,
#     text=f"Độ chính xác mô hình: {accuracy:.2f}",
#     font=("Arial", 14),
#     justify="left",
# )
# info_label.pack(pady=20)

frame = cus.CTkFrame(root, border_width=2)
frame.pack(pady=20, padx=20)

values = {}
for i, col in enumerate(X_train.columns):
    label = cus.CTkLabel(frame, text=col, font=("Arial", 12))
    label.grid(row=i, column=0, padx=10, pady=5, sticky="w")
    entry = cus.CTkEntry(frame, width=200)
    entry.grid(row=i, column=1, padx=10, pady=5)
    values[col] = entry

# Hàm dự đoán
def predict():
    input_data = np.array([[float(values[col].get()) for col in X_train.columns]])
    prediction = voting_clf.predict(input_data)[0]
    result_text = "Có nguy cơ mắc bệnh" if prediction == 1 else "Không có nguy cơ mắc bệnh"
    result_label.configure(text=f"Kết quả dự đoán: {result_text}")

#Nút dự đoán và hiển thị kết quả
predictBtn = cus.CTkButton(frame, text="Dự đoán", command=predict)
predictBtn.grid(column=4, row=9, padx=10)
#Hiển thị kết quả
result_label = cus.CTkLabel(frame, text="", font=("Time New Roman", 14, "bold"))
result_label.grid(column=4, row=10, pady=5, padx=5)

root.mainloop()